# Parse ATL08 HDF File 
1. This notebook is designed to parse ATL08 HDF file And convert to CSV and Shapefile (actually Geopackage!!)  
2. Quick data inspection and visualization 

Bidhya N Yadav   
Oct 17, 2020  

In [ ]:
%matplotlib inline

In [ ]:
import os

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.figsize': (15, 8), 'font.size': 10})

from shapely.geometry import Point#, Polygon, mapping
import h5py
from astropy.time import Time

import hvplot
import hvplot.pandas
import geoviews as gv

## Add code to download data
If you need to download data   


In [ ]:
# Setup path to where you have downloaded ATL08 HDF files
# path = f'D:/icesat2/download' #path = './download' test_ATL08
path = '../../../download' #change to to suit your path

## Parser to Convert ATL08 HDF file to CSV and Geopackage/Shapefile

In [ ]:
def gps2dyr(time, offset = 0):
    """ Converte GPS time to decimal years. Helper function"""
    time = time + offset
    gps_time = Time(time, format='gps')#.decimalyear
    iso_time = Time(gps_time, format='iso')
    iso_time = iso_time.value
    dt = pd.to_datetime(iso_time)
    return dt

def read_atl08(icesat2_path, gis_output='shp'):
    """ Read ATL08 HDF file and convert to csv and geopackage.     
        Extract variables of interest and separate the ATL08 file 
        into each beam (ground track) and ascending/descending orbits.
        TODO: Append ascending/descending nodes to output name
        
        All the hdf files inside the specified folder will be parsed
        Save in the same location
    """
    files = os.listdir(icesat2_path)
    hdf_files = [f for f in files if f.endswith('.h5') and 'ATL08' in f]
    print(f'Number of HDF files: {len(hdf_files)}')
    for f in hdf_files:
        print(f'Processing HDF file: {f}')
        hdf_path = f'{icesat2_path}/{f}'
        res_dict = {}
        meta_dict = {} #These will hold metadata required for scalars per ground-track
        group = ['gt1l', 'gt1r', 'gt2l', 'gt2r', 'gt3l', 'gt3r']
        qual_str_count = ''
        with h5py.File(hdf_path, 'r') as fi:
            # subset group based on data
            group = [g for g in list(fi.keys()) if g in group]
            group1 = len(group)
            group = [g for g in group if 'land_segments' in fi[f'/{g}']]
            group2 = len(group)
            if group2<group1:
                print.info(f'Non-empty groups: {group2}/{group1}')
            # NB: Assert if at least one group present else may be error due to enumeration
            if len(group) == 0:
                print(f'No ground track data in this file: {f}')
                continue
            t_ref = fi['/ancillary_data/atlas_sdp_gps_epoch'][:] #scalar 1 value
            for k,g in enumerate(group):
                # 1) Read in data for a single beam #
                lat = fi[f'/{g}/land_segments/latitude']
                lon = fi[f'/{g}/land_segments/longitude']
                t_dt = fi[f'/{g}/land_segments/delta_time']
                layer_flag = fi[f'/{g}/land_segments/layer_flag']
                # Extract everything for terrain and canopy variables
                terrain_keys = fi[f'{g}/land_segments/terrain'].keys()
                terrain_keys = list(terrain_keys)
                terrain_dict = {}
                for tk in terrain_keys:
                    terrain_dict[tk] = fi[f'{g}/land_segments/terrain/{tk}']
                #terrain = pd.DataFrame.from_dict(terrain_dict)
                # Do the same for Canopy
                canopy_keys = fi[f'{g}/land_segments/canopy'].keys()
                canopy_keys = list(canopy_keys)
                # Remove these two keys as they contain some multivalue tuples which are just no-data in version 002
                canopy_keys.remove('canopy_h_metrics')
                canopy_keys.remove('canopy_h_metrics_abs')
                canopy_dict = {}
                for ck in canopy_keys:
                    canopy_dict[ck] = fi[f'{g}/land_segments/canopy/{ck}']                    
                # Remove the canopy subset flag that seems added in version 3 because this is an array but we can't save array to shapefile
                if 'subset_can_flag' in canopy_dict:
                    del canopy_dict['subset_can_flag']
                if 'subset_te_flag' in terrain_dict:
                    del terrain_dict['subset_te_flag']
                #Merge two dictionaries (order should be retained; verify when running again wity Python 3.7)
                terrain_dict.update(canopy_dict)
                
                # 2) To Make Pandas dataframe
                # Collect everythin into one dictionary
                gt_dict = {'lon':lon, 'lat':lat, 't_dt':t_dt, 'layer_flag':layer_flag}
                gt_dict.update(terrain_dict)
                #df = pd.DataFrame({'lon':lon, 'lat':lat, 'h_li': h_li, 'q_flag':q_flag, 't_dt':t_dt})
                df = pd.DataFrame.from_dict(gt_dict)
                nan_value =  fi[f'/{g}/land_segments/terrain/h_te_mean'].attrs['_FillValue'][0]
                df= df.replace(nan_value, np.nan)

                #Convert GPS time to actual time using function
                df['t_dt'] = df['t_dt'].apply(gps2dyr, offset=t_ref[0])

                all_points = len(df)
                if len(df)>0:
                    # Assemble ground track into a dictionary, later we convert to csv and shp through df
                    res_dict[g] = df
            #----------------------------------------------------------------------------------------------
            # Now that ATL08 data from separate ground tracks are in one dict, merge it to df and save to csv/shp
            if len(res_dict)>0:
                # To guard againt empty result dictionary created with no icesat2 passing the quality control
                # 1. Combine Dataframes for each of 6 ground-tracks into single Dataframe
                count = 0
                for k in res_dict.keys():
                    if count == 0:
                        df = res_dict[k]
                        df['strip'] = k
                        count += 1
                    else:
                        df1 = res_dict[k]
                        df1['strip'] = k
                        df = pd.concat([df, df1], axis=0)
                # Choose filename for csv and shapefile
                atl_fname = os.path.splitext(hdf_path)[0].split('/')[-1]
                df.to_csv(f'{icesat2_path}/{atl_fname}.csv', index=False)
                
                # 2. Convert to Geopandas
                df['geometry'] = df[['lon', 'lat']].apply(lambda x: Point(x), axis=1)
                gdf = gpd.GeoDataFrame(df, geometry='geometry', crs = 'epsg:4326')
                if gis_output=='shp':
                    gdf['t_dt'] = gdf['t_dt'].dt.strftime('%Y-%m-%d %H:%M:%S.%f') #To prevent DriverSupportError: ESRI Shapefile does not support datetime fields
                    gdf.to_file(f'{icesat2_path}/{atl_fname}.shp')
                elif gis_output=='gpkg':
                    gdf.to_file(f'{icesat2_path}/{atl_fname}.gpkg', driver='GPKG') #ignore fiona error for now
            else:
                print(f"\tNo Ground Track in this HDF file; csv or shp not created")

In [ ]:
# Call the above function to parse hdf file
print(f'Parsing ATL08 hdf file')
read_atl08(path, gis_output='gpkg')

---

# Inspect the parsed csv and gis files

In [ ]:
path = '../../../download' #change to to suit your path
files = os.listdir(f'{path}')

hdf_files = [f for f in files if f.endswith('.h5') and 'ATL08' in f]
shp_files = [f for f in files if f.endswith('.gpkg') and 'ATL08' in f]
csv_files = [f for f in files if f.endswith('.csv') and 'ATL08' in f]

print('No. of shp files',len(shp_files), len(hdf_files))
files

In [ ]:
# Load csv and gis files
fname ='processed_ATL08_20190731112820_05100403_003_01' #shp_files[idx].split('.gpkg')[0]
df = pd.read_csv(f'{path}/{fname}.csv', parse_dates=True) # Load CSV data
df['t_dt'] = pd.to_datetime(df.t_dt) # Convert to pandas native date time format to plotting libraries understand the data

gdf = gpd.read_file(f'{path}/{fname}.gpkg', parse_dates=True) # Load GIS file
# Get unique reference ground tracks
gtls = list(df.strip.unique())
print(len(df), gtls)

In [ ]:
df.head()

In [ ]:
# gdf.plot()
gdf.head()

In [ ]:
# Pick one Ground Track for interactive visualization example
df = df[df.strip == 'gt2l']

## Interactive Visualization of Parse Files
Because the output files are csv and geopackage you are use any software or GIS program for analysis and visualization. For, now lets exploit the rich Python ecosystem (primarily hvPlot) for this quick visualization to get a sense of our data

Map Composing  
    - Overlay : *  
    - Tile : +  

In [ ]:
base = gv.tile_sources.ESRI # Add basemap to get sense of where the data is
gtracks = gdf.hvplot.points(geo=True, color='strip', alpha=0.7, width=500, height=700) # Plot ground tracks
# gtracks = gdf[gdf.strip=='gt2l'].hvplot.points(geo=True, color='strip', alpha=0.7, width=500, height=700) # Or subset a particular track

## Plot terrain and canopy data
terrain = df.hvplot(y='lat', x='h_te_min', kind='scatter', width=350, height=650, color='brown', s=20, alpha=.9).relabel('Terrain')
canopy = df.hvplot(y='lat', x='h_max_canopy_abs', kind='scatter', width=350, height=650, color='green', s=10, alpha=.9, title=f'Elevation', xlabel='meters').relabel('Canopy')
# Compose a Plot with with basemap overlaying ground track and another tile with terrain and canopy 
base * gtracks + terrain * canopy